# 1. Overview

This is a Demo project for the Matrix Profiling Anomaly Detection (MPAD) Framework.

In this Demo project, we present the flow of the framework of using Matrix Profile (MP) approach for detecting the anomalies in time series data. The Matrix Profile measures the similarity joint between one time series with itself or two time series. The data in this similarity joint (i.e., the MP) could be leveraged to identify the similarities and differences, which is used in anomaly detection tasks in this Demo project.

The MPAD Framework includes **5 steps** as shown in the framework below:

*(Left side of the figure shows the steps and the flow, and Right side of the figure shows the functions and variables developed in this demo project.)*
<img src="imgs/flow_functions.png" width="1200"/>


### Step 1 - Data Extraction for the units of analysis

The ***raw time series list*** benchmark_ts and evaluate_ts, and the MilePost (MP) int value are inputs of the Function `dataset_extraction()`, and be used to generate the list of desired ***segments from the time series***, including seg_benchmark_mp,seg_evaluate_mp, and their normalized arrays: seg_benchmark_mp_scaled,seg_evaluate_mp_scaled.

- Function `ts_normalize()` is called in `dataset_extraction()` to take in a time series dataframe ts and transform it into the mean-normlaized version


### Step 2 - Generate matrix profile

The ***segments from the time series*** with benchmark data and evaluation data are then processed in the Function `mp_score()` to generate the ***matrix profiles***. For given time series, using a sliding window, sub-sequences from the time series are extracted and the distance between the extracted sub-sequence and the rest of the entire time series are calculated.

- Function `sliding_window()` is called in `mp_score()` to take in a time series data and generate a 2D (w_l by n-w_l) array of sliced data using sliding window method.


### Step 3 - Anomaly detection (peaks detection)

The ***matrix profiles*** array (matrix_profile) and the time series dataframe being eveluated (evaluate_ts), limit on the number of peaks (top_peaks) into the Function `select_peak()` to generate the ***output*** with selected peaks (peaks across all channels) and the matrix profile scores of the selected peaks. 

- `mp_find_peaks()` is called in Function `select_peak()` with tolerance as the distance constraint for peaks to generate the array of identified peaks.

- It is worth noting that the output can be varied based on the matrix profile or post-processing matrix profile approxmy, including: 1) Directly using the matrix profile; 2) Median filtering of matrix profiles; 3) Approximating the matrix profile by a peicewise linear regression.

### Step 4 - Multi-channel data integration

The ***output*** with selected peaks from individual channels and MPs (including peaks from each MP+channel combination) are merged and processed in Function `multi_ch_peak_detection()` and turned into arrarys of ***integrated selected peaks*** and their associated mp-score per channel.

- Function `multi_ch_kde()` is called in `multi_ch_peak_detection()` to generate the selected anomlaies as an array, as well as generate the kde pdf.


### Step 5 - Performance evaluation

Users can either directly use Functions `ad_score_macro()` & `ad_score_micro()` to analyze the ***performance (recall and precision)*** of the anomaly detection ***output*** from individual channels, or from the ***integrated selected peaks***.

- ad_score_macro retruns a dict of recall and precision for each channel
- ad_score_micro returns recall and precision for each MP-channel, therefore, it is called micro. 
- The micro version also handles cases where the denominator of recall or precision is zero.

# 2. Load Packages

Import the following required libraries to build the environment before running the Demo project.

*(There is a GPU-related pacakge that is optional for users)*

In [1]:
# pip install -r requirements.txt

In [2]:
# Basic Data Analysis Packages
import pandas as pd
import numpy as np
from numpy import array, linspace
from random import seed, gauss
import os
import pickle
from sklearn.neighbors import KernelDensity
import pwlf #The package for peicewise linear regression, users may need to install the pwlf first before import
# pip install pwlf

# Scipy-related Packages
import scipy as sp
from scipy import *
from scipy.signal import *

# Plot-related Packages
from matplotlib import pyplot as plt
from matplotlib.pyplot import plot
# Customize matpolib rc (runtime configuration) settings 
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams.update({'font.size': 15})


# GPU-related Package
# import cupy as cp
# This package is used for faster data processing with GPU, which requires NVIDIA CUDA GPU. Users can use CPU option instead.
# See Details in (https://docs.cupy.dev/en/stable/overview.html)

# 3. Functions Block

All the functions developed for the MPAD Framework are presented in this block.

## 3.1 Data Extraction Functions

Functions used for data extraction for the units of analysis.

In [ ]:
def dataset_extraction(benchmark_ts,evaluate_ts,MP,seg_len):

    '''
    This function process the raw time series list and extract out the list of desired segments 
    from the time series: seg_benchmark_mp,seg_evaluate_mp, 
    along with their normalized arrays: seg_benchmark_mp_scaled, seg_devaluate_mp_scaled.
    
    Parameters
    ----------
    *benchmark_ts: dataframe
        The benchmark time series dataframe read from the csv file.
    *evaluate_ts:
        The evaluate time series dataframe read from the csv file.
    *MP:
        The MilePost index number.
    *seg_len:
        The segmentation length index. (default is 1)
            
    Returns
    -------
    *seg_benchmark_mp: array
        Segmented benchmark time series.
    *seg_evaluate_mp: array
        Segmented evaluate time series.
    *seg_benchmark_mp_scaled: array
        Normalized segmented benchmark time series.
    *seg_evaluate_mp_scaled: array
        Normalized segmented evaluate time series.
    '''
    
    # isoltaing the data for segments (e.g., one mile post)
    seg_benchmark_mp=benchmark_ts[(benchmark_ts["Full MP"] >= MP) & (benchmark_ts["Full MP"] < (MP+seg_len))]
    seg_benchmark_mp=seg_benchmark_mp.set_index('Full MP')
    seg_evaluate_mp=evaluate_ts[(evaluate_ts["Full MP"] >= MP) &  (evaluate_ts["Full MP"] < (MP+seg_len))]
    seg_evaluate_mp=seg_evaluate_mp.set_index('Full MP')
    
    # copy the dataframes to be used for normlization
    seg_benchmark_mp_scaled =seg_benchmark_mp.copy()
    seg_evaluate_mp_scaled =seg_evaluate_mp.copy()
    
    # apply zero-mean normalization
    seg_benchmark_mp_scaled =ts_normalize(seg_benchmark_mp_scaled)
    seg_evaluate_mp_scaled =ts_normalize(seg_evaluate_mp_scaled)
    seg_benchmark_mp_scaled=seg_benchmark_mp_scaled.to_numpy()
    seg_evaluate_mp_scaled=seg_evaluate_mp_scaled.to_numpy()
    
    return seg_benchmark_mp,seg_evaluate_mp,seg_benchmark_mp_scaled,seg_evaluate_mp_scaled

def ts_normalize(ts):

    '''
    This function takes in a time series dataframe ts, returns the mean-normlaized version
    
    Parameters
    ----------
    *ts: dataframe
        The input time series dataframe.
            
    Returns
    -------
    *ts: dataframe
        Mean-normalized version of the time series.
    '''
    
    for column in ts.columns:
        ts [column] = (ts[column] - ts[column].mean())
    return ts

def defect_pro(defect_list):
    
    '''
    This function gets a list of defects [loc, severity, profile] 
    and returns the list of defects for different constraint.
    
    Parameters
    ----------
    *defect_list: list
        List of the defects.
            
    Returns
    -------
    *defect_loc: list
        List of defects for different constraint.
    '''
    
    defect_loc=[]
    for i in defect_list:
        defect_loc.append(i[0])
    return defect_loc

## 3.2 Matrix Profile Quantification Functions

Functions used to generate matrix profiles.

In [ ]:
def mp_score(benchmark,evaluation,feature_w,gpu_flag=0):
    
    '''
    This function takes the benchmark and evaluation time series and returns matrix profile.
    There are two options of matrix profile processing: using GPU or CPU
    
    Parameters
    ----------
    *benchmark: array
        Segmented (or normalized segemen) benchmark time series.
    *evaluation: array
        Segmented (or normalized segemen) evaluate time series.
    *feature_w: int
        Sliding window size.
    *gpu_flag: int
        Flag to choose use gpu or not for processing matrix profile. (default is 1)
            
    Returns
    -------
    *cp.asnumpy(g_mp).flatten(): array
        Processed matrix profile.
    *mp.flatten(): array
        Processed matrix profile.
    '''
    
    if gpu_flag:
        # Transfering data to GPU
        g_benchmark=cp.asarray(sliding_window(benchmark,feature_w))
        g_evaluation=cp.asarray(sliding_window(evaluation,feature_w))

        # initializing MP on GPU
        g_mp=cp.zeros([1, g_evaluation.shape[1]], dtype = float)
        for i in range(g_evaluation.shape[1]):
            X=cp.tile(g_evaluation[:,[i]],g_benchmark.shape[1])
            g_mp[0,i]=cp.min(cp.linalg.norm(cp.subtract(g_benchmark,X), ord=2, axis=0, keepdims=False))

        return cp.asnumpy(g_mp).flatten()
    else:
        benchmark=sliding_window(benchmark,feature_w)
        evaluation=sliding_window(evaluation,feature_w)
        mp=np.zeros([1, evaluation.shape[1]], dtype = float)

        for i in range(evaluation.shape[1]):
            X=np.tile(evaluation[:,[i]],benchmark.shape[1])
            mp[0,i]=np.min(np.linalg.norm(np.subtract(benchmark,X), ord=2, axis=0, keepdims=False))

        return mp.flatten()

def sliding_window(ts,w_l):
    
    '''
    This function takes a time series (ts), and a window size (w_l), 
    and returns a 2D (w_l by n-w_l) array of sliced data using sliding window method.
    
    Parameters
    ----------
    *ts: array
        The input time series array.
    *w_l: int
        Sliding window size.
            
    Returns
    -------
    *outcome_ts: array
        Processed sliding window time series.
    '''
    
    outcome_ts = np.transpose(np.lib.stride_tricks.sliding_window_view(ts.flatten(), w_l))
    
    return outcome_ts

## 3.3 Anomaly (Peak) Detection Functions

Functions that focuses on processing matrix profile for anomaly (peak) detection.

In [ ]:
def mp_find_peaks(mp_proxy,evaluate_ts,tolerance):
    
    '''
    This function takes in a matrix profile proxy and identifies the peaks on the proxy with a predefined tolerance,
    then outputs the identified peaks, their mp values, and mile-values. 
    
    Matrix profile (mp) proxy is an apprxomation of the mp. For example the mp could be
    approximated with a peicewise linear function or a median filter to reduce noise.
    
    Parameters
    ----------
    *mp_proxy: array
        Matric profile proxy array.
    *evaluate_ts: array
        Evaluate time series array.
    *tolerance: int
        The distance constraint for peaks detection.
        
    Returns
    -------
    *peaks: array
        Array of the selected peaks on matrix profile proxy.
    *peak_values: array
        Arrary of the mp_proxy function corresponding to the peaks' locations.
    *mile_values: array
        The peak locations in miles.
    '''
    
    peaks, _ = find_peaks(mp_proxy,height=np.histogram(mp_proxy,bins=21)[1][11],distance=tolerance)
    peak_values=mp_proxy[peaks]
    mile_values=evaluate_ts.index.values[peaks]
    return peaks, peak_values, mile_values

def select_peak(matrix_profile,mp_ch_key,evaluate_ts,top_peaks,tolerance,reg_flag,med_filter_flag,plot_flag,defect_loc_list):
    
    '''
    This function select the peaks based on the matrix profiles, and then return the selected peaks.
    
    Parameters
    ----------
    *matrix_profile: array
        Generated matrix profile.
    *mp_ch_key: string
        Name of the specific channel.
    *evaluate_ts: array
        Evaluate time series array.
    *top_peaks: int
        Number of top peaks selected among all the peaks that are returned from a matrix peofile or its proxy.
    *tolerance: int
        The minimum distance between peaks for the find_peaks function.
    *reg_flag: int
        Decides whether to use peicewise linear regression approximation of matrix profile.
    *med_filter_flag:
        Decides whether to use a median filter on matrix profile.
    *plot_flag:
        Decides whether to plot peaks on matrix profiles.
    *defect_loc_list: list
        List of ground truth defects/anomalies.
            
    Returns
    -------
    *output: list
        List of results with different output combinations.
    **selected_peaks: array
        Array of the selected peaks.
    **selected_peaks_mp_values: array
        Array of matrix profile score of the selected peaks.
    '''
    
    peaks, peak_values, mile_values= mp_find_peaks(matrix_profile,evaluate_ts,tolerance)
    # mp_peak_values=matrix_profile[peaks]
    temp=[]
    w=10
    mpl=len(matrix_profile)-1
    for i in peaks:
        f1=matrix_profile[min(i+w,mpl)]-matrix_profile[i]
        f2=matrix_profile[i]-matrix_profile[max(0,i-w)]
        if f1*f2<0:
            temp.append(i)
        
    mp_peak_values=np.array(matrix_profile[temp])
    mile_values=evaluate_ts.index.values[temp]
    
    # Here the mile values are sorted based on the matrix profile scores
    mile_values_sorted=mile_values[np.argsort(mp_peak_values)[::-1]]
    mp_peak_values_sorted=mp_peak_values[np.argsort(mp_peak_values)[::-1]]
    
    selected_peaks=mile_values_sorted[:top_peaks]
    selected_peaks_mp_values=mp_peak_values_sorted[:top_peaks]
    
    # --------------- finding and processing peaks on processed matrix profile ---------------------------------
 
    if med_filter_flag:
        w_ma=251 # for medfilter this should be an odd integer
        # Median filtering of matrix profiles
        mp_smoothed=sp.signal.medfilt(matrix_profile, kernel_size=w_ma)
        # Selecting peaks on the smoothed profile
        peaks_smoothed,mp_peak_values_smoothed,mile_values_smoothed= mp_find_peaks(mp_smoothed,evaluate_ts,tolerance)
    
    if reg_flag:
        x=evaluate_ts.index.values[:len(matrix_profile)]
        # Approximating the matrix profile by a peicewise linear regression
        my_pwlf = pwlf.PiecewiseLinFit(x, matrix_profile)
        breaks = my_pwlf.fit(10) # This need to be adjusted by users.
        mp_pwlf=my_pwlf.predict(x)
        peaks_pwlf, mp_peak_values_pwlf,mile_values_pwlf= mp_find_peaks(mp_pwlf,evaluate_ts,tolerance)
    
    
    if plot_flag:
        font = {'family' : 'Helvetica',
        'weight' : 'regular',
        'size'   : 12}

        plt.rcParams['figure.figsize'] = 12, 4
        plt.rc('font', **font)
        fig, ax = plt.subplots()
        x=evaluate_ts.index.values[:len(matrix_profile)]; intc=0 # This is to fix the problem with issues at the begining of a mile post - change it to 2 to fix that
        # This is plotting matrix profiles versus miles
        plt.plot((x)[intc:len(matrix_profile)],matrix_profile[intc:len(matrix_profile)],color='k',linewidth=2,alpha=0.2, label='Matrix Profile')
        # Plotting the top_peaks peak points
        plt.scatter(selected_peaks,selected_peaks_mp_values,color='r',label='Top Predicted Peaks')
        
        if med_filter_flag:
            plt.plot(x,mp_smoothed,label='Med. Fil. MP')
            plt.scatter(mile_values_smoothed,mp_peak_values_smoothed,color='c',marker='^',s=100, label='Med. Fil. Peaks')

        if reg_flag:
            plt.plot(x,mp_pwlf,label='PWLF MP')
            plt.scatter(mile_values_pwlf,mp_peak_values_pwlf,color='g',marker='H',label='PWLF Peaks')
        
        plt.scatter(np.array(defect_loc_list),np.random.uniform(min(matrix_profile), max(matrix_profile), size = (1,len(defect_loc_list))),color='b',marker='x',s=100,label='Ground Truth (Actual) Anomalies')
        plt.legend()
        plt.xlabel('Mile Post')
        plt.ylabel('Matrix Profile Score')
        plt.savefig('Graph_'+mp_ch_key+".pdf", bbox_inches='tight')
        plt.savefig('Graph_'+mp_ch_key+".svg", bbox_inches='tight', format='svg')
        plt.show()
 

    # Preparing the output as a list depending on post-processing of matrix profile
    if reg_flag and med_filter_flag:
        output=[selected_peaks, selected_peaks_mp_values,mile_values_smoothed,mp_peak_values_smoothed,
            mile_values_pwlf, mp_peak_values_pwlf]
    elif med_filter_flag:
        output=[selected_peaks, selected_peaks_mp_values,mile_values_smoothed,mp_peak_values_smoothed]
    elif reg_flag:
        output=[selected_peaks, selected_peaks_mp_values,mile_values_pwlf,mp_peak_values_pwlf]
    else:
        output=[selected_peaks, selected_peaks_mp_values]
        
    return output

## 3.4 Multi-channel Data Integration Functions

Functions that focuses on getting peaks from indiviudal channels and combines them through kernel density estimation (kde) and peak detection on kde probability distribution function (pdf).

In [ ]:
def multi_ch_kde(ch_peaks,ch_peaks_scores,MP,kde_res,bw,tolerance,mode='weighted'):

    '''
    This function takes in peaks selected across m channels and use weighted/uniform mode to 
    combines them through kernel density estimation (kde) and 
    peak detection on kde probability distribution function (pdf)
    
    Parameters
    ----------
    *ch_peaks: array
        Peaks selected across m channels.
    *ch_peaks_scores: array
        Matrix profile score for each peak.
    *MP:
        Mile post index.
    *kde_res: int
        The kde span resolution.
    *bw: int
        The kde kernel bandwidth, which is used to create a probability distirbution function from the data points.
    *tolerance: int
        The distance constraint for peaks detection.
    *mode: string
        The mode for selection, weighted/uniform.
            
    Returns
    -------
    *selected_anomalies: array
        Selected anomalies.
    *mp_ticks:
        Reference for the Mile Posts.
    *pdf: array
        Kde probability distribution function.
    *peaks: array
        Peaks identified in kde pdf.
    '''
    
    ch_peaks_arr=ch_peaks.reshape(-1,1)

    if mode=="weighted":
        print("weighted KDE _________")
        kde=KernelDensity(kernel='gaussian',bandwidth=bw).fit(ch_peaks_arr,sample_weight=ch_peaks_scores)
    else:
        print("uniform KDE _________")
        kde=KernelDensity(kernel='gaussian',bandwidth=bw).fit(ch_peaks_arr)
        
    mp_ticks = np.linspace(MP,MP+1,num=kde_res)
    kde_scores = kde.score_samples(mp_ticks.reshape(-1,1))
    pdf=np.exp(kde_scores)
    peaks, _ = find_peaks(pdf,height=np.histogram(pdf,bins=5)[1][2])
    selected_anomalies=mp_ticks[peaks]
    
    return selected_anomalies,mp_ticks,pdf,peaks

def multi_ch_peak_detection(peaks_dict,MP_list,attributes,kde_res,kde_bw,kde_tolerance,defect_dict,text_flag=0):

    '''
    This function takes in peak_dict (a dictionary that includes peaks data from each MP+channel combination) 
    and other variables to process them into arrays of peaks and their associated mp-score per channel.
    
    Parameters
    ----------
    *peaks_dict: dictionary
        Dictionary that includes peaks data from each MP+channel combination.
    *MP_list: list
        List of the Mipleposts being evaluated.
    *attributes: list
        Channels to be considered in the multi-channel peak detection.
    *kde_res: int
        The kde span resolution.
    *kde_bw：int
        The kde kernel bandwidth.
    *kde_tolerance: int
        The distance constraint for peaks detection.
    *defect_dict：dictionary
        Dictionary of the ground truth defects.
    *text_flag: 
        Decides whether to print the peak selection results and ground truth defects.
           
            
    Returns
    -------
    *kde_peaks_dict: dictionary
        Dictionary of the peaks detected through kde.
    *pdf: array
        kernel density estimation - probability distribution function.
    '''
    
    kde_peaks_dict={}
    
    # Processing the peaks for kde
    for MP in MP_list:
        temp_peaks=np.array([])
        temp_mp_scores=np.array([])
        for x in attributes:
            # extracting the peaks and their mp scores for each MP
            temp=peaks_dict[str(MP)+'-'+str(x)]
            temp_peaks=np.concatenate([temp_peaks,temp[0]])
            temp_mp_scores=np.concatenate([temp_mp_scores,temp[1]])               
        
        selected_anomalies,mp_ticks,pdf,peaks=multi_ch_kde(temp_peaks,temp_mp_scores/max(temp_mp_scores)*100,MP,kde_res,kde_bw,kde_tolerance,mode='weighted')
        kde_peaks_dict[str(MP)+'-multi']=[selected_anomalies]

        # Prints the predicted and actual anomalies
        if text_flag:
            print("KDE Anomalies:",np.around(selected_anomalies,3))
            print("Ground Truth:",defect_dict[MP])

            
        font = {'family' : 'Helvetica',
        'weight' : 'regular',
        'size'   : 12}

        plt.rc('font', **font)
        fig, ax = plt.subplots()
        ax.set_xlim(MP,MP+1)
        ax.plot(mp_ticks, pdf/max(pdf), linewidth=3, alpha=0.9, scalex=True,label='KDE pdf | bw='+str(kde_bw))
        for i in temp_peaks:
            if i==temp_peaks[len(temp_peaks)-1]:
                ax.axvline(i,ymin=0, ymax=1,c='k', alpha=0.1,label='Detected Peaks on Channels')
            else:
                ax.axvline(i,ymin=0, ymax=1,c='k', alpha=0.1)
        ax.scatter(np.array(defect_pro(defect_dict[MP])), np.random.rand(len(defect_pro(defect_dict[MP]))), fc='red', alpha=1,label='Ground Truth (Actual) Anomalies')
        ax.scatter(np.array(selected_anomalies), (pdf/max(pdf))[peaks], fc='g', alpha=1,marker='x',s=150,label='PDF Peaks')
        #         ax.legend(loc='upper right')
        fig.set_figwidth(8)
        fig.set_figheight(3)
        plt.ylabel('Density',fontsize=10)
        plt.xlabel('Mile Post',fontsize=10)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.legend(prop={"size":10})
        plt.savefig("plot_"+str(MP)+".pdf",bbox_inches='tight')
        plt.show()

    return kde_peaks_dict,pdf

## 3.5 Performance Evaluation Functions

Functions that focuses on performance evaluation. 

Two functions are very similar: 
- `ad_score_macro` retruns a dict of recall and precision for each channel.
- `ad_score_micro` returns recall and precision for each MP-channel, therefore, it is called micro. The micro version also handles cases where the denominator of recall or precision is zero.


In [ ]:
def ad_score_macro(peaks_dict,MP_list,attributes,inspection,threshold,top_peaks,defect_dict):
    
    '''
    This function measures the performance of anomaly detection using common metrics (i.e., tp, fp, fn) 
    and retruns all the basic performance metrics in the form of a dictionary
    and calculates and plots recall and precision for different data channels.
    
    It returns a dictionary of recall and precision for all mileposts for each channel, hence macro.
    
    Parameters
    ----------
    *peaks_dict: dictionary
        Dictionary of the identified peaks.
    *MP_list: list
        List of the Mipleposts being evaluated.
    *attributes: list
        Channels to be considered in evaluation.
    *inspection: list
        Inspections being evaluated.
    *threshold：float
        miles - this is the vicinity tolerance threshold to calculate the performance 
        - tolerance of distance b/w predicted and GT anomalies.
    *top_peaks：int
        The number of top peaks selected among all the peaks that are returned from a matrix peofile or its proxy.
    *defect_dict：dictionary
        Ground truth defects dictionary.
            
    Returns
    -------
    *scores_dict: dictionary
        Dictionary of the performance evaluation results.
    '''
    
    scores_dict={} # Initailizing the dictionary for evaluation metrics
    
    for x in attributes:
        tp=0;tp_2=0;fn=0;fp=0 # Initalizing basic metrics
        
        for MP in MP_list:
            gt_count=len(defect_dict[MP]) # Getting the total number of grougd truth anomalies
            temp=str(MP)+'-'+str(x)

            for j in defect_pro(defect_dict[MP]):
                dis=np.absolute(np.array(peaks_dict[temp][0])-j) # Calculating the distance between predicted and actual anomalies
                if dis.size>0 and min(dis)<threshold:
                    tp+=1
                else:
                    fn+=1
                    
            if gt_count==0:
                    fp+=len(peaks_dict[temp][0]) # all detected peaks will be false positives
            else:
                for k in peaks_dict[temp][0]:
                    dis=np.absolute(np.array(defect_pro(defect_dict[MP]))-k)

                    if dis.size>0 and min(dis)>threshold:
                        fp+=1
                    if dis.size>0 and min(dis)<threshold:
                        tp_2+=1

        #print(x,tp,fn,fp) #Uncomments to see the values for each channel   

        mp_metrics=np.array([round(tp/(tp+fn),2),round(tp/(tp+fp),2)])
        scores_dict[x]=mp_metrics

    #------- Output Block ---------
    print('--------------------------------------------------------------------------------------------------')
    print('The >>',inspection.upper(),'<< inspection')
    print('Detection tolerance:>>',threshold,'<< miles')
    print('Selecting the top >>',top_peaks,'<< peaks')
    print()
    # print('Atrribute','tp','fn','fp')      
    scores_df=pd.DataFrame.from_dict(scores_dict)
    scores_df=scores_df.rename(index={0: 'Recall',1: 'Precision'})
    scores_df
    scores_df.T.plot.bar()
#     scores_df.T.plot.scatter(x='Recall', y='Precision', s=50)
#     for idx, row in scores_df.T.iterrows(): 
#         plt.text(row['Recall']+gauss(0, 0.05), row['Precision']+gauss(0, 0.03), idx,fontsize=10)
    plt.show()
    return scores_dict

def ad_score_micro(peaks_dict,MP_list,attributes,threshold,defect_dict):
    
    '''
    This function measures the performance of anomaly detection using common metrics (i.e., tp, fp, fn) 
    and retruns all the basic performance metrics in the form of a dictionary
    and calculates and plots recall and precision for different data channels.
    
    It returns a dictionary of recall and precision for all mileposts-channel keys, hence micro.
    Micro handles the cases where denominator of recall and/or precion is zero.
    
    Parameters
    ----------
    *peaks_dict: dictionary
        Dictionary of the identified peaks.
    *MP_list: list
        List of the Mipleposts being evaluated.
    *attributes: list
        Channels to be considered in evaluation.
    *threshold: float
        miles - this is the vicinity tolerance threshold to calculate the performance 
        - tolerance of distance b/w predicted and GT anomalies.
    *defect_dict：dictionary
        Ground truth defects dictionary.
            
    Returns
    -------
    *scores_dict: dictionary
        Dictionary of the performance evaluation results.
    '''
    
    scores_dict={}

    for MP in MP_list:
        gt_count=len(defect_dict[MP])
        
        for x in attributes:
            temp=str(MP)+'-'+str(x)
            tp=0;tp_2=0;fp=0;fn=0

            for j in defect_pro(defect_dict[MP]): #[0],<<<<=======================
                dis=np.absolute(np.array(peaks_dict[temp][0])-j)

                if dis.size>0 and min(dis)<threshold:
                    tp+=1
                else:
                    fn+=1
            
            if gt_count==0:
                fp+=len(peaks_dict[temp][0]) # all detected peaks will be false positives
                
            for k in peaks_dict[temp][0]:
                dis=np.absolute(np.array(defect_pro(defect_dict[MP]))-k)
                if dis.size>0 and min(dis)>threshold:
                    fp+=1
                if dis.size>0 and min(dis)<threshold:
                    tp_2+=1 #Have not used this in showing the outcome

            if tp+fp!=0 and tp+fn!=0:
                scores_dict[temp]=[len(peaks_dict[temp][0]),gt_count,tp,fp,fn,round(tp/(tp+fn),2),round(tp/(tp+fp),2)]
            elif tp+fp==0:
                scores_dict[temp]=[len(peaks_dict[temp][0]),gt_count,tp,fp,fn,round(tp/(tp+fn),2),'na']
            elif tp+fn==0:
                scores_dict[temp]=[len(peaks_dict[temp][0]),gt_count,tp,fp,fn,'na',round(tp/(tp+fp),2)]
            else:
                scores_dict[temp]=[len(peaks_dict[temp][0]),gt_count,tp,fp,fn,'na',round(tp/(tp+fp),2)]
    scores_dict
    return scores_dict

## 3.6 Experiments Functions

Function that process the calculated performance of different data channels.

In [ ]:
def top_performing_channels(geo_flag,inspection,vicinity_threshold,top_peaks,scores_dict,top_rows):
    
    '''
    This function focuses on processing the performance of different data channels.
    
    Parameters
    ----------
    *geo_flag: int
        Decides to use acceleration versus geometry channels.
    *inspection: list
        Inspections being evaluated.
    *attributes: list
        Channels to be considered in evaluation.
    *vicinity_threshold: float
        Vicinity tolerance threshold to calculate the performance.
    *top_peaks：int
        The number of top peaks selected among all the peaks.
    *scores_dict：dictionary
        Dictionary of the performance evaluation results.
    *top_rows：int
        Number of top rows to be selected and show.
            
    Returns
    -------
    *scores_df.head(top_rows): dictionary
        Dictionary of the sorted performance evaluation results with selected top rows.
    '''
    
    if geo_flag:
        print("Evaluating -- GEOMETRY -- channel for the >>",inspection,"<< inspection")
    else:
        print("Evaluating -- Acceleration -- channel for the >>",inspection,"<< inspection")
    print("Performance threshold:",vicinity_threshold,"  miles")
    print('Selecting the >>',top_peaks,'<< peaks')
    print("")

    scores_df=pd.DataFrame.from_dict(scores_dict)
    scores_df=scores_df.T.sort_values(by=[0], ascending=False).rename(columns={0: 'Recall',1: 'Precision'})
    scores_df
    
    return scores_df.head(top_rows)


# 4. Demo Project

This is the demo of the MPAD with an anomaly detection task and some sample datasets. In this task, we are detecting anomalies on a railroad time series data, in which the time series data are acceleration and geometry sensor readings across a part of railroad. In this demo project we show the two geometry sensor reading as channels: *Right Profile 62, Left Profile*.


***Note that sample data are not provided, and the data below are dummy examples for this Demo Project, and can't directly run and generate outputs.***

***Users can customize their own time series datasets and adjust the parameters according to their needs.***

## 4.1 Groud Truth Anomalies Data

Ground truth data - specific to data sets 

The data should be adjusted for each project - The format of the ground truth dictionary is as presnted below.

Similar dictionaries should be populated (either manual or through code) for a datset that is evaluated through this code.

In [ ]:
# Example for ppopulating the ground truth data assuming two inspections with two miles posts 1 and 2
# The right side is a dictionary key input | left side a list of lists. Each one of the lists include ground truth anomlay location, level, and channel

# First Inspection
defect_dict_1={}
defect_dict_1[1]=[[1.15,3,'Left Prof'],[1.65,2,'Right Profile 62']]
defect_dict_1[2]=[[1.62,1,'Right Profile']]

# Second Inspection
defect_dict_2={}
defect_dict_2[1]=[[1.63,2,'Left Prof']]
defect_dict_2[2]=[] # representing no defect on this milepost.

defect_dict_list=[defect_dict_1,defect_dict_2]

## 4.2 Project Parameters

Users need to customize their project parameters on their specific tasks and datasets.

In [ ]:
#--------------Project Specific Data---------------------------------#

# Data Channels
accelerations=['Your proper channel names']
geometry=['Right Prof 62','Left Profile'] # Example shown for this demo project

# Mipleposts being evaluated
MP_list=[1] # Example shown for this demo project

# Inspections being evaluated 
inspections=['first','second'] # Example shown for this demo project
ins_inx=1

# Selecting the defect dictionary for plotting
defect_dict=defect_dict_list[ins_inx]

#---------------------------------------------------------------------#


#--------------Parameters----------------------------------------------#

##-------------Commonly Changed Parameters----------------------------##
gpu_flag=0    # Using GPU for matrix profile calculation - default is 0
geo_flag=1    # changing b/w geometry and acceleration channels
top_peaks=3   # peak selection and processing parameters

# Evaluation Parameter
vicinity_threshold=0.03

# Change attribute as per implementation
if geo_flag:
    attributes=geometry
else:
    attributes=accelerations[0:1]
##--------------------------------------------------------------------##

on_off_falg=1 # This is the flag that turns matrix profile calculations on or off
feature_w=75  # sliding window size
ma_flag=0     # parameters to apply a median filter to matrix profile to reduce noise
bw=0.01       # The kernel bandwidth

reg_flag=0          # Decides whether to use peicewise linear regression approximation of matrix profile
med_filter_flag=1   # Decides whether to use a median filter on matrix profile
plot_flag=1         # Decides whether to plot peaks on matrix profiles

kde_res=1000        # The kde span resolution
kde_bw=0.02         # The kde bandwidth is used to create a probability distirbution function from the data points
ps_tolerance=120    # The minimum distance between peaks for the find_peaks function
kde_tolerance=kde_res/20 # The distance constraint for peaks detection
    
matrix_profile_dict={} # In case of storing the matrix profile dictionary; not used in this demo project

#----------------------------------------------------------------------#

## 4.3 Import Dataset

Users need to change this into their customized time series datasets as benchmark and evaluation time series.

In [ ]:
# Extracting the data from the files for becnhmark and evaluation inspections---
if attributes==geometry:
    benchmark_ts= pd.read_csv('Your Benchmark Geometry Time Series File.csv')
    evaluation_ts=pd.read_csv('Your Evaluation Geometry Time Series File.csv')  
else:
    benchmark_ts= pd.read_csv('Your Benchmark Acceleration Time Series File.csv')
    evaluation_ts= pd.read_csv('Your Benchmark Acceleration Time Series File.csv')

## 4.4 Matrix Profiling & Anomaly Detection

The main class of this demo project.

In [ ]:
# Initializing the dictionary to store peaks and their associated matrix profile scores and locations in miles
ch_peaks_dict={}

# Going through list of mileposts being evaluated, form matrix-profiles, and detect all peaks/anomalies for each channel
for MP in MP_list:
    for x in attributes:
        benchmark_ts_x=benchmark_ts[['Location of Your time series',x]]
        evaluation_ts_x=evaluation_ts[['Location of Your time series',x]]

        benchmark_ts_mp,evaluation_ts_mp,benchmark_ts_mp_scaled,evaluation_ts_mp_scaled=dataset_extraction(benchmark_ts_x,evaluation_ts_x,MP,1)

        # The main matrix profile computation ----------------------------------------------------
        if on_off_falg:
            matrix_profile=mp_score(benchmark_ts_mp_scaled,evaluation_ts_mp_scaled,feature_w,gpu_flag=0)
        else:
            matrix_profile# Implement the code to load calculated matrix profiles
        #--------------------------------Single Channel Peak Detection -------------------------------
        key=str(MP)+'-'+str(x)
        # Once you generate the image delete key2 line and replace key2 with key in print
        key2=str(MP)+'-'+str(x) 
        print(key2)
        output=select_peak(matrix_profile,key,evaluation_ts_mp,top_peaks,ps_tolerance,reg_flag,med_filter_flag,plot_flag,defect_pro(defect_dict[MP]))
        ch_peaks_dict[key]=output # Storing the data for multi channel assessments

**Example Outputs:**

--------------------------------------------------------------------------------------------------

1-Right Prof 62
<img src="imgs/RightProf_peaks.png" width="1200"/>

1-Left Profile
<img src="imgs/LeftProf_peaks.png" width="1200"/>

## 4.5 Performance Evaluation

Evaluating the performance of individual channels in detecting the anomalies

In [ ]:
scores_dict=ad_score_macro(ch_peaks_dict,MP_list,attributes,inspections[ins_inx],vicinity_threshold,top_peaks,defect_dict_list[ins_inx])

print('==================================================================================================')
print('\033[1m' +'Dictionary of Performance Metrics for Channels' + '\033[0m')
pd.DataFrame.from_dict(scores_dict).head()

**Example Outputs:**

--------------------------------------------------------------------------------------------------

The >> SECOND << inspection

Detection tolerance:>> 0.03 << miles

Selecting the top >> 3 << peaks

<img src="imgs/individual_cha_perf.png" width="1200"/>

==================================================================================================

**Dictionary of Performance Metrics for Channels**

|  | Recall | Precision |
| --- | --- | --- |
| Right Prof 62 | 1.0 | 1.0 |
| Left Profile | 1.0 | 0.5 |

In [ ]:
# Sorting channels based on their recall value
print('\033[1m' +'Sorted based on channel recall' + '\033[0m')
top_rows=len(geometry)
top_per_ch=top_performing_channels(geo_flag,inspections[ins_inx],vicinity_threshold,top_peaks,scores_dict,top_rows)
top_per_ch

**Example Outputs:**

--------------------------------------------------------------------------------------------------


**Sorted based on channel recall**

Evaluating -- GEOMETRY -- channel for the >> second << inspection

Performance threshold: 0.03   miles

Selecting the >> 3 << peaks

|  | Recall | Precision |
| --- | --- | --- |
| Right Prof 62 | 1.0 | 1.0 |
| Left Profile | 1.0 | 0.5 |

## 4.6 Multi-Channel Data Integration

Ensembling all peaks/anomalies into a kde pdf and detect probabilistic anomalies on the pdf

In [ ]:
print('\033[1m' + 'Ensemble Analysis Using Kernel Density Estimation' + '\033[0m')
multi_ch_peaks_dict,pdf=multi_ch_peak_detection(ch_peaks_dict,MP_list,attributes,kde_res,kde_bw,kde_tolerance,defect_dict)

**Example Outputs:**

--------------------------------------------------------------------------------------------------

**Ensemble Analysis Using Kernel Density Estimation**

weighted KDE _________

<img src="imgs/weighted_kde.png" width="800"/>

In [ ]:
# Evaluating the performance of multiple channels in detecting all exceptions
scores_dict_kde=ad_score_macro(multi_ch_peaks_dict,MP_list,['multi'],inspections[ins_inx],vicinity_threshold,top_peaks,defect_dict_list[ins_inx])

print('==================================================================================================')
print('\033[1m' +'Dictionary of Aggregated Performance Metrics' + '\033[0m')
print(pd.DataFrame.from_dict(scores_dict_kde).T.rename(columns={0: 'Recall',1: 'Precision'}))

**Example Outputs:**

--------------------------------------------------------------------------------------------------

The >> SECOND << inspection

Detection tolerance:>> 0.03 << miles

Selecting the top >> 3 << peaks

<img src="imgs/multi_cha_perf.png" width="800"/>

==================================================================================================

**Dictionary of Aggregated Performance Metrics**

|  | Recall | Precision |
| --- | --- | --- |
| multi | 1.0 | 1.0 |